# Env Configuration

In [15]:
from google.colab import drive
drive.mount('drive', force_remount=True)
!pip install keras_metrics

Mounted at drive


## Setup

In [16]:
import os
import time
import gc
import numpy as np
import pandas as pd
from scipy import interp, stats
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
from keras import backend as K
from keras import optimizers
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Dropout, Input, concatenate
from keras.losses import binary_crossentropy
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model, model_to_dot
import keras_metrics as km
import math
import itertools
import re
import os
import imageio
from scipy.ndimage import rotate
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt

'''fixed settings'''
rotations_of_benignant    = 5
rotations_of_malignant    = 13
slices_per_nodule         = 5
number_of_folds           = 10
strategy                  = 'first'
repeat_slices             = False
base_dir                  = "/content/drive/My Drive/Master's degree/Research/data/"
images_dir                = base_dir + "images/solid-nodules-with-attributes/"
features_path             = base_dir + "features/solidNodules.csv"
images_resolution         = 64
metrics                   = {'acc': [], 'spec': [], 'sens': [], 'f1_score': [], 'auc': []}
number_of_validations     = 1
drop_random_values_gen    = stats.norm(scale=0.05, loc=0.4)
conv1_search_space        = [32, 48, 64, 96]
dense1_search_space       = [32, 64, 128]
dense2_search_space       = [16, 24, 32]

'''Flexible settings'''
input_shape = (64, 64, 5, 1)

feature_injection    = True
layer_to_be_injected = 'flatten' #flatten, dense1, dense2
features_set         = 'all'     #all, opt, margin, shape 
experiment_name      = layer_to_be_injected + ' - ' + features_set if (feature_injection)  else 'none'

def feat_input_shape():
  if (features_set == 'all'):
    return (71,)
  elif (features_set == 'opt'):
    return (38,)
  elif (features_set == 'margin'):
    return (12,)
  elif (features_set == 'shape'):
    return (8,)
  
def feat_input_subset():
  if (features_set == 'opt'):
    return optimized_features_set
  elif (features_set == 'margin'):
    return margin_features_set
  elif (features_set == 'shape'):
    return shape_features_set

feature_input_shape    = feat_input_shape()
shape_features_set     = [16, 17, 18, 19, 20, 21, 22, 23]
margin_features_set    = [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
optimized_features_set = [2, 3, 4, 6, 9, 11, 12, 18, 19, 22, 26, 28, 29, 
                             30, 31, 32, 33, 35, 38, 40, 41, 44, 46, 47, 48, 
                             50, 53, 54, 56, 57, 59, 60, 62, 63, 65, 67, 71, 72]
  
print("Feature injection: "    + str(feature_injection))
print("Layer to be injected: " + layer_to_be_injected)
print("Features set: "         + features_set)

Feature injection: True
Layer to be injected: flatten
Features set: all


## Normalize getting the first slices (Unchanging)
Function that normalize getting the first slices

In [0]:
def normalize_first(nodules, n_slices, repeat=False):
    '''Normalizes the nodule slices number:
    - A nodule with less than n slices is completed with black slices
    - A nodule with more than n slices have its n first slices selected'''
    
    normalized_slices = []

    for nodule in nodules:
        new_nodule = []

        if repeat:
            times = math.ceil(n_slices/len(nodule))
            nodule = list(itertools.chain.from_iterable(itertools.repeat(x, times) for x in nodule))

        if len(nodule) <= n_slices:
                for slice in nodule:
                    new_nodule.append(slice)
                for i in range(n_slices - len(nodule)):
                    new_nodule.append(np.zeros((images_resolution, images_resolution)))
        elif len(nodule) > n_slices:
            for i in range(0, n_slices):
                new_nodule.append(nodule[i])
        normalized_slices.append(new_nodule)
    return normalized_slices

## Read images
Function to read images from files and returns a list of numpy

In [0]:
def read_images(path, path_features):
    '''Reads the images files in our file structure and mounts an array
    Parameters:
        path (string): path to the nodules folders
        path_features (string): path to the features .csv
    Returns:
        list: list of nodules with slices as Numpy Arrays
        features: list of features corresponding to the nodules on list'''

    df = pd.read_csv(path_features)
    
    scaler = MinMaxScaler(copy=False)
    df[df.columns[2:74]] = scaler.fit_transform(df[df.columns[2:74]])
    
    allFeatures = df.values

    lista       = []
    features    = []

    for _, dirs, _ in os.walk(path):
        for dirname in sorted(dirs, key=str.lower):
            for _, dirs1, _ in os.walk(path + "/" + dirname):
                for dirname1 in sorted(dirs1, key=str.lower):
                    for root2, _, files2 in os.walk(path + "/" + dirname + "/" + dirname1):
                        slices = []
                        files2[:] = [re.findall('\d+', x)[0] for x in files2]

                        axis = 0 # To get the Rows indices
                        examColumn = 0 # Column of the csv where the exam code is
                        noduleColumn = 1 # Column of the csv where the nodule code is

                        # index of the rows that have the exam id equal to the exam id of the current nodule
                        indExam  = np.where(allFeatures[:,examColumn] == dirname)[axis]

                        # index of the rows that have the nodule id equal to the id of the current nodule
                        indNodule = np.where(allFeatures[:,noduleColumn] == dirname1)[axis]

                        i = np.intersect1d(indExam,indNodule)

                        # A list are returned, but there's just one value, so I used its index
                        index = 0
                        exam = allFeatures[i,examColumn][index]
                        nodule = allFeatures[i,noduleColumn][index]

                        '''Verify if there's more than one index for each nodule
                        and if there's divergence between the nodule location and the
                        csv values'''

                        if((len(i) > 1) or (str(exam) != str(dirname)) or (str(nodule) != str(dirname1))):
                            print("Features error!")
                        else:
                            '''Transform the list of index with just one value in a
                            primitive value to use as index to save the features values'''
                            i = i[0]

                        for f in sorted(files2, key=float):
                            img = imageio.imread(root2 + "/" + f + ".png", as_gray=True)
                            slices.append(img)

                        lista.append(slices)
                        
                        if (features_set == 'all'):
                          features.append(allFeatures[i,2:73].tolist())
                        else:
                          features.append(allFeatures[i,feat_input_subset()].tolist())

    return lista, features

## My Kfold
k_folder made to get balanced data between benigno and maligno

In [0]:
def my_kfold(ben, mal, f_ben, f_mal, n_splits, ben_rot, mal_rot):
    kf = KFold(n_splits)
    
    f_mal_train, f_mal_test = [], []
    mal_train, mal_test = [], []
    for train_index, test_index in kf.split(mal):
        mal_train.append([mal[index] for index in train_index])
        f_mal_train.append([f_mal[index] for index in train_index])

        mal_test.append([mal[index] for index in test_index])
        f_mal_test.append([f_mal[index] for index in test_index])

    ben_train, ben_test = [], []
    f_ben_train, f_ben_test = [], []
    
    # percorro o mal_test para que os folds de test tenham o mesmo número de itens
    for (train_index, test_index), mal_te, mal_tr in zip(kf.split(ben), mal_test, mal_train):
        
        sample = np.random.choice(test_index, len(mal_te), replace=False)
        sample_ = np.setdiff1d(test_index, sample)

        ben_train_ind = np.concatenate((train_index, sample_))

        '''This line guarantees that the ben and mal train batches are the same size'''
        ben_train_ind = np.random.choice(ben_train_ind, len(mal_tr), replace=False)

        ben_train.append([ben[index] for index in ben_train_ind])
        f_ben_train.append([f_ben[index] for index in ben_train_ind])
        
        ben_test.append([ben[index] for index in sample])
        f_ben_test.append([f_ben[index] for index in sample])

    X_test, Y_test = [], []
    for b, m in zip(ben_test, mal_test):
        X_test.append(np.concatenate((b, m), 0))

        y_test = len(b) * [0] + len(m) * [1]
        Y_test.append(np.array(y_test))

    f_test = []
    for f_b, f_m in zip(f_ben_test, f_mal_test):
        
        f_test.append(np.concatenate((f_b, f_m), 0))

    X_train, Y_train = [], []
    f_train = []
    for i in tqdm(range(n_splits)):
        print("INDEX: ", i)
        print("ben_train: ", len(ben_train[i]))
        print("mal_train: ", len(mal_train[i]))
        print("ben_test: ",  len(ben_test[i]))
        print("mal_test: ",  len(mal_test[i]))
        
        b, m = ben_train[i], mal_train[i]
        f_b_train, f_m_train = f_ben_train[i], f_mal_train[i]

        X_train.append(np.concatenate((b, m), 0))
        f_train.append(np.concatenate((f_b_train, f_m_train), 0))

        y_train = len(b) * [0] + len(m) * [1]
        Y_train.append(np.array(y_train))

    return X_train, X_test, f_train, f_test, Y_train, Y_test

## Get folds
Function that is called to get the folds of the cross validation

In [0]:
def get_folds(basedir, n_slices, strategy='first', repeat=False, features=None):
    ben_dir = basedir + "benigno"
    mal_dir = basedir + "maligno"

    ben, f_ben = read_images(ben_dir, features)
    mal, f_mal = read_images(mal_dir, features)

    if strategy == 'first':
        ben = normalize_first(ben, n_slices, repeat)
        mal = normalize_first(mal, n_slices, repeat)

    ben = np.concatenate(ben).reshape(len(ben), n_slices, images_resolution, images_resolution, 1)
    mal = np.concatenate(mal).reshape(len(mal), n_slices, images_resolution, images_resolution, 1)

    ben = np.moveaxis(ben, 1, 3)
    mal = np.moveaxis(mal, 1, 3)

    ben_zip = list(zip(ben, f_ben))
    np.random.shuffle(ben_zip)
    ben, f_ben = zip(*ben_zip)

    mal_zip = list(zip(mal, f_mal))
    np.random.shuffle(mal_zip)
    mal, f_mal = zip(*mal_zip)

    X_train, X_test, f_train, f_test, Y_train, Y_test = my_kfold(ben, mal, f_ben, 
                                                                 f_mal, number_of_folds, 
                                                                 rotations_of_benignant, 
                                                                 rotations_of_malignant)

    return X_train, X_test, f_train, f_test, Y_train, Y_test

# Valition code


In [0]:
def get_model(conv1, dense1, dense2, drop1, drop2):
    input_layer = Input(input_shape)
    
    if (feature_injection):
      print("(features input creation) Feature injection ACTIVATED - " + features_set)
      input_features_layer = Input(feature_input_shape)
    else:
      print("Feature injection DEACTIVATED")
      
    conv_layer1    = Conv3D(filters=conv1, kernel_size=(3, 3, 3), 
                        activation='relu')(input_layer)
    pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer1)
    
    flatten_layer  = Flatten()(pooling_layer1)
    
    if (feature_injection and layer_to_be_injected == 'flatten'):
      print("Injecting - " + features_set + " - on Flatten!")
      flatten_layer = concatenate([flatten_layer, input_features_layer])

    dense_layer1 = Dense(units=dense1, activation='relu')(flatten_layer)
    
    if (feature_injection and layer_to_be_injected == 'dense1'):
      print("Injecting - " + features_set + " - on Dense1!")
      dense_layer1 = concatenate([dense_layer1, input_features_layer])
    
    dense_layer1 = Dropout(drop1)(dense_layer1)

    dense_layer2 = Dense(units=dense2, activation='relu')(dense_layer1)
    
    if (feature_injection and layer_to_be_injected == 'dense2'):
      print("Injecting - " + features_set + " - on Dense2!")
      dense_layer2 = concatenate([dense_layer2, input_features_layer])
    
    dense_layer2 = Dropout(drop2)(dense_layer2)

    output_layer = Dense(units=1, activation='sigmoid')(dense_layer2)
    
    model = []
    if (feature_injection):
      print("(model creation) Feature - " + features_set + " - injection ACTIVATED")
      model = Model(inputs=[input_layer, input_features_layer], outputs=output_layer)
    else:
      print("(model creation) Feature injection DEACTIVATED")
      model = Model(inputs=input_layer, outputs=output_layer)
      
    opt = optimizers.RMSprop(lr=0.0001)

    model.compile(loss=binary_crossentropy, optimizer=opt, 
    metrics=['accuracy', km.binary_true_positive(), km.binary_true_negative(), 
                km.binary_false_positive(), km.binary_false_negative(), 
                km.binary_f1_score()])

    return model

## Cross-validation

In [0]:
start = time.time()
for i in range(number_of_validations):
    X_train_, X_test_, f_train_, f_test_, Y_train_, Y_test_=  get_folds(basedir=images_dir, 
                                                                        n_slices=slices_per_nodule, 
                                                                        strategy=strategy, 
                                                                        repeat=repeat_slices,
                                                                        features=features_path)
    i = 0
    best_run = {}
    for i in range(30):
      print("Run :" + str(i))
      conv1  = np.random.choice(conv1_search_space, size=1)[0]
      dense1 = np.random.choice(dense1_search_space, size=1)[0]
      dense2 = np.random.choice(dense2_search_space, size=1)[0]
      
      drop1 = -1
      while (drop1 < 0 or drop1 > 0.5):
        drop1 = drop_random_values_gen.rvs()
        
      drop2 = -1
      while (drop2 < 0 or drop2 > 0.5):
        drop2 = drop_random_values_gen.rvs()
      
      m = {'acc': [], 'auc': []}

      model = get_model(conv1, dense1, dense2, drop1, drop2)

      for X_train, X_test, f_train, f_test, Y_train, Y_test in zip(X_train_, X_test_, f_train_, f_test_, Y_train_, Y_test_):
        if (feature_injection):
          print("(fitting) Feature - " + features_set + " - injection ACTIVATED")
          model.fit([X_train, f_train], Y_train, batch_size=128, epochs=10, verbose=0)
        else:
          print("(fitting) Feature injection DEACTIVATED")
          model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=0)
          
        scores = []
        if (feature_injection):
          print("(evaluating) Feature - " + features_set + " - injection ACTIVATED")
          scores = model.evaluate([X_test, f_test], Y_test, verbose=0)
        else:
          print("(evaluating) Feature injection DEACTIVATED")
          scores = model.evaluate(X_test, Y_test, verbose=0)
          
        tp, tn, fp, fn = scores[2], scores[3], scores[4], scores[5]
        
        acc = scores[1]*100
        
        # AUC
        pred = []
        
        if (feature_injection):
          print("(predicting) Feature - " + features_set + " - injection ACTIVATED")
          pred = model.predict([X_test, f_test]).ravel()
        else:
          print("(predicting) Feature injection DEACTIVATED")
          pred = model.predict(X_test).ravel()
        
        fpr, tpr, thresholds_keras = roc_curve(Y_test, pred)
        auc_val = auc(fpr, tpr)
        
        print('Test accuracy: ', acc)
        print('Test AUC: ', auc_val)

        m['acc'].append(acc)
        m['auc'].append(auc_val)
      
      run_output = {'acc': np.mean(m['acc']), 'auc': np.mean(m['auc']), 'conv1': conv1, 'dense1': dense1, 'dense2': dense2, 'drop1': drop1, 'drop2': drop2}
      print(run_output)
      
      if (len(best_run) == 0 or run_output['auc'] > best_run['auc']):
        best_run = run_output
      elif (run_output['auc'] == best_run['auc'] and run_output['acc'] > best_run['acc']):
        best_run = run_output

end = time.time()

print("Time on optimization:", (end - start)/60, "minutes")

print()
print('Summary of the best model: -----------------------')
print(best_run)
c1 = best_run['conv1']
d1 = best_run['dense1']
d2 = best_run['dense2']
dr1 = best_run['drop1']
dr2 = best_run['drop2']

print('Conv 1:', c1, 'unidades')
print('Dense 1:', d1, 'unidades')
print('Dense 2:', d2, 'unidades')
print('Dropout 1:', dr1)
print('Dropout 2:', dr2)

 20%|██        | 2/10 [00:00<00:00, 14.74it/s]

INDEX:  0
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  1
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  2
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  

 40%|████      | 4/10 [00:00<00:00, 14.64it/s]

3
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  4
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  5
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27


 80%|████████  | 8/10 [00:00<00:00, 14.66it/s]

INDEX:  6
ben_train:  240
mal_train:  240
ben_test:  27
mal_test:  27
INDEX:  7
ben_train:  241
mal_train:  241
ben_test:  26
mal_test:  26
INDEX:  8
ben_train:  241
mal_train:  241
ben_test:  26
mal_test:  26


100%|██████████| 10/10 [00:00<00:00, 14.66it/s]


INDEX:  9
ben_train:  241
mal_train:  241
ben_test:  26
mal_test:  26
Run :0
(features input creation) Feature injection ACTIVATED - all
Injecting - all - on Flatten!
(model creation) Feature - all - injection ACTIVATED
(fitting) Feature - all - injection ACTIVATED
(evaluating) Feature - all - injection ACTIVATED
(predicting) Feature - all - injection ACTIVATED
Test accuracy:  85.1851862889749
Test AUC:  0.9012345679012346
(fitting) Feature - all - injection ACTIVATED
(evaluating) Feature - all - injection ACTIVATED
(predicting) Feature - all - injection ACTIVATED
Test accuracy:  77.77777821929367
Test AUC:  0.8895747599451302
(fitting) Feature - all - injection ACTIVATED
(evaluating) Feature - all - injection ACTIVATED
(predicting) Feature - all - injection ACTIVATED
Test accuracy:  87.03703681627908
Test AUC:  0.9293552812071331
(fitting) Feature - all - injection ACTIVATED
(evaluating) Feature - all - injection ACTIVATED
(predicting) Feature - all - injection ACTIVATED
Test accuracy